In [1]:
import numpy as np

### Data I/O

In [58]:
data = open("shakespeare.txt", "r").read()
chars = list(set(text))
print(chars)
print(len(data))
data_size, vocab_size = len(data), len(chars)
char_to_ix = {ch:i for i, ch in enumerate(char)}
ix_to_char = {i:ch for i, ch in enumerate(char)}
char_to_ix

['U', 'F', 'h', 'n', 'z', '$', 'O', 'Q', 'Y', 'I', 's', 'c', 'K', 'G', 'H', '.', 'e', "'", 'P', ',', 'v', 'Z', 'X', 'M', ':', 'f', 'A', 'd', 'J', '3', 'u', 'y', 'B', 'm', 'i', 'j', 'x', 'o', '-', 'l', 'D', 't', 'N', 'V', 'T', '!', '[', 'w', 'C', 'S', '&', 'E', ' ', 'k', 'g', 'r', 'L', 'a', 'W', '?', 'R', 'b', ';', '\n', 'q', 'p', ']']
4573338


{'U': 0,
 'F': 1,
 'h': 2,
 'n': 3,
 'z': 4,
 '$': 5,
 'O': 6,
 'Q': 7,
 'Y': 8,
 'I': 9,
 's': 10,
 'c': 11,
 'K': 12,
 'G': 13,
 'H': 14,
 '.': 15,
 'e': 16,
 "'": 17,
 'P': 18,
 ',': 19,
 'v': 20,
 'Z': 21,
 'X': 22,
 'M': 23,
 ':': 24,
 'f': 25,
 'A': 26,
 'd': 27,
 'J': 28,
 '3': 29,
 'u': 30,
 'y': 31,
 'B': 32,
 'm': 33,
 'i': 34,
 'j': 35,
 'x': 36,
 'o': 37,
 '-': 38,
 'l': 39,
 'D': 40,
 't': 41,
 'N': 42,
 'V': 43,
 'T': 44,
 '!': 45,
 '[': 46,
 'w': 47,
 'C': 48,
 'S': 49,
 '&': 50,
 'E': 51,
 ' ': 52,
 'k': 53,
 'g': 54,
 'r': 55,
 'L': 56,
 'a': 57,
 'W': 58,
 '?': 59,
 'R': 60,
 'b': 61,
 ';': 62,
 '\n': 63,
 'q': 64,
 'p': 65,
 ']': 66}

### Hyperparameter

In [59]:
hidden_size = 100
seq_length = 25
learning_rate = 1e-1

### Model Parameter

In [60]:
Wxh = np.random.randn(hidden_size, vocab_size) * 0.01
Whh = np.random.randn(hidden_size, hidden_size) * 0.01
Why = np.random.randn(vocab_size, hidden_size) * 0.01
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

In [63]:
p = 0
print(data[p:p+seq_length])
print(data[p+1:p+seq_length+1])
inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
print (inputs)
print (targets)


First Citizen:
Before we 
irst Citizen:
Before we p
[1, 34, 55, 10, 41, 52, 48, 34, 41, 34, 4, 16, 3, 24, 63, 32, 16, 25, 37, 55, 16, 52, 47, 16, 52]
[34, 55, 10, 41, 52, 48, 34, 41, 34, 4, 16, 3, 24, 63, 32, 16, 25, 37, 55, 16, 52, 47, 16, 52, 65]


In [44]:
print(inputs)

NameError: name 'inputs' is not defined